# Monitor ML runs live

<a target="_blank" href="https://colab.research.google.com/github/neptune-ai/examples/blob/main/how-to-guides/monitor-ml-runs/notebooks/Monitor_ML_runs_live.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/>
</a>

## Introduction

This guide will show you how to:

* Monitor training and evaluation metrics and losses live
* Monitor hardware resources during training

By the end of it, you will monitor your metrics, losses, and hardware live in Neptune!

## Before you start

This notebook example lets you try out Neptune as an anonymous user, with zero setup.

If you want to see the example logged to your own workspace instead:

  1. Create a Neptune account. [Register &rarr;](https://neptune.ai/register)
  1. Create a Neptune project that you will use for tracking metadata. For instructions, see [Creating a project](https://docs.neptune.ai/setup/creating_project) in the Neptune docs.

## Install Neptune and dependencies

In [5]:
! pip install -U neptune tensorflow

  Obtaining dependency information for tensorflow from https://files.pythonhosted.org/packages/ba/7c/b971f2485155917ecdcebb210e021e36a6b65457394590be01cc61515310/tensorflow-2.13.0-cp310-cp310-win_amd64.whl.metadata
  Using cached tensorflow-2.13.0-cp310-cp310-win_amd64.whl.metadata (2.6 kB)
  Obtaining dependency information for tensorflow-intel==2.13.0 from https://files.pythonhosted.org/packages/40/fa/98115f6fe4d92e1962f549917be2dc8e369853b7e404191996fedaaf4dd6/tensorflow_intel-2.13.0-cp310-cp310-win_amd64.whl.metadata
  Using cached tensorflow_intel-2.13.0-cp310-cp310-win_amd64.whl.metadata (4.1 kB)
  Using cached tensorboard-2.13.0-py3-none-any.whl (5.6 MB)
  Using cached google_auth_oauthlib-1.0.0-py2.py3-none-any.whl (18 kB)
  Obtaining dependency information for google-auth<3,>=1.6.3 from https://files.pythonhosted.org/packages/9c/8d/bff87fc722553a5691d8514da5523c23547f3894189ba03b57592e37bdc2/google_auth-2.22.0-py2.py3-none-any.whl.metadata
  Using cached google_auth-2.22.0-py2

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-api-python-client 2.86.0 requires uritemplate<5,>=3.0.1, which is not installed.
tensorflow-gpu 2.10.1 requires keras<2.11,>=2.10.0, but you have keras 2.13.1 which is incompatible.
tensorflow-gpu 2.10.1 requires protobuf<3.20,>=3.9.2, but you have protobuf 4.24.0 which is incompatible.
tensorflow-gpu 2.10.1 requires tensorboard<2.11,>=2.10, but you have tensorboard 2.13.0 which is incompatible.
tensorflow-gpu 2.10.1 requires tensorflow-estimator<2.11,>=2.10.0, but you have tensorflow-estimator 2.13.0 which is incompatible.


## Create a basic training script

As an example I'll use a script that trains a Keras model on the MNIST dataset.

In [8]:
from tensorflow import keras

# parameters
params = {
    "epoch_nr": 100,
    "batch_size": 256,
    "lr": 0.005,
    "momentum": 0.4,
    "use_nesterov": True,
    "unit_nr": 256,
    "dropout": 0.05,
}

mnist = keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

model = keras.models.Sequential(
    [
        keras.layers.Flatten(),
        keras.layers.Dense(params["unit_nr"], activation=keras.activations.relu),
        keras.layers.Dropout(params["dropout"]),
        keras.layers.Dense(10, activation=keras.activations.softmax),
    ]
)

optimizer = keras.optimizers.SGD(
    learning_rate=params["lr"],
    momentum=params["momentum"],
    nesterov=params["use_nesterov"],
)

model.compile(optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"])

## Initialize Neptune and create new run

To create a new run for tracking the metadata, you tell Neptune who you are (`api_token`) and where to send the data (`project`).

You can use the default code cell below to create an anonymous run in a public project. **Note**: Public projects are cleaned regularly, so anonymous runs are only stored temporarily.

### Log to your own project instead

Replace the code below with the following:

```python
import neptune
from getpass import getpass

run = neptune.init_run(
    project="workspace-name/project-name",  # replace with your own (see instructions below)
    api_token=getpass("Enter your Neptune API token: "),
    capture_hardware_metrics=True,
    capture_stderr=True,
    capture_stdout=True,
)
```

To find your API token and full project name:

1. [Log in to Neptune](https://app.neptune.ai/).
1. In the bottom-left corner, expand your user menu and select **Get your API token**.
1. The workspace name is displayed in the top-left corner of the app.

    To copy the project path, in the top-right corner, open the settings menu and select **Properties**.

For more help, see [Setting Neptune credentials](https://docs.neptune.ai/setup/setting_credentials) in the Neptune docs.

In [6]:
import neptune

run = neptune.init_run(
    project="ibm26ashish/mnistdemo",
    api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiJjOTViNGNjZi00ZDJhLTQxNjYtYmE4ZS0xYWRiNzc0ZWQ3MWEifQ==",
    capture_hardware_metrics=True,
    capture_stderr=True,
    capture_stdout=True,
)  # Hardware metrics, stderr, and stdout are not captured by default in interactive kernels

https://app.neptune.ai/ibm26ashish/mnistdemo/e/MNIS-2


**To open the run in the Neptune web app, click the link that appeared in the cell output.**

We'll use the `run` object we just created to log metadata. You'll see the metadata appear in the app.

## Add logging for metrics and losses

Since we are using Keras we'll create a Callback that logs metrics and losses after every epoch.

In [7]:
class NeptuneLogger(keras.callbacks.Callback):
    def on_batch_end(self, batch, logs={}):
        for log_name, log_value in logs.items():
            run[f"batch/{log_name}"].append(log_value)

    def on_epoch_end(self, epoch, logs={}):
        for log_name, log_value in logs.items():
            run[f"epoch/{log_name}"].append(log_value)

We need to pass it to the `callbacks` argument.

In [9]:
model.fit(
    x_train,
    y_train,
    epochs=params["epoch_nr"],
    batch_size=params["batch_size"],
    callbacks=[NeptuneLogger()],
)

Epoch 1/100
235/235 [==============================] - 3s 9ms/step - loss: 1.6315 - accuracy: 0.5987
Epoch 2/100
235/235 [==============================] - 2s 10ms/step - loss: 0.8790 - accuracy: 0.8134
Epoch 3/100
235/235 [==============================] - 3s 11ms/step - loss: 0.6460 - accuracy: 0.8475
Epoch 4/100
235/235 [==============================] - 3s 11ms/step - loss: 0.5447 - accuracy: 0.8641
Epoch 5/100
235/235 [==============================] - 2s 10ms/step - loss: 0.4864 - accuracy: 0.8741
Epoch 6/100
235/235 [==============================] - 3s 11ms/step - loss: 0.4491 - accuracy: 0.8809
Epoch 7/100
235/235 [==============================] - 2s 10ms/step - loss: 0.4228 - accuracy: 0.8867
Epoch 8/100
235/235 [==============================] - 3s 11ms/step - loss: 0.4025 - accuracy: 0.8906
Epoch 9/100
235/235 [==============================] - 3s 11ms/step - loss: 0.3862 - accuracy: 0.8944
Epoch 10/100
235/235 [==============================] - 2s 10ms/step - loss: 0.3732

## Stop logging

Once you are done logging, stop tracking the run.

In [5]:
run.stop()

Shutting down background jobs, please wait a moment...
Done!
Waiting for the remaining 46 operations to synchronize with Neptune. Do not kill this process.
All 46 operations synced, thanks for waiting!
Explore the metadata in the Neptune app:
https://app.neptune.ai/ibm26ashish/mnistdemo/e/MNIS-1/metadata


## See results live in the UI

* The logs will be visible as charts in the **Charts** dashboard on the left
* Neptune also logs your system hardware consumption. These are visible under the **Monitoring** dashboard